# Ventilation Analysis
This notebook runs the ventilation analysis script and displays the results.

In [ ]:
!python ventilation_analysis.py

## Results
The analysis results are saved in `ventilation_report.md` and visualizations are saved as images.